In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quick start with Model Garden - CXR Foundation

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgoogle-health%2Fcxr-foundation%2Fmaster%2Fnotebooks%2Fquick_start_with_model_garden.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/cxr-foundation/blob/master/notebooks/quick_start_with_model_garden.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying CXR Foundation to Vertex AI and making online or batch predictions to get embeddings from chest X-ray images or text.

Vertex AI makes it easy to serve your model and make it accessible to the world. Learn more about [Vertex AI]((https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform).


### Objectives

- Deploy CXR Foundation to a Vertex AI Endpoint and get online predictions.
- Upload CXR Foundation to Vertex AI Model Registry and get batch predictions.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Import packages and define common functions

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

import datetime
import importlib
import io
import json
import os
import uuid

import numpy as np
from google.cloud import aiplatform, storage
from google.oauth2 import credentials
from PIL import Image

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

models, endpoints = {}, {}


def download_gcs_image_bytes(gcs_uri, creds):
    """Download an image from Cloud Storage."""
    storage_client = storage.Client(credentials=creds)
    blob = storage.blob.Blob.from_string(gcs_uri, client=storage_client)
    return blob.download_as_bytes()

In [ ]:
# @title Set up Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. Make sure that you have the following required roles:
# @markdown - [Storage Admin](https://cloud.google.com/iam/docs/understanding-roles#storage.admin) (`roles/storage.admin`) to create and use Cloud Storage buckets
# @markdown - [Service Usage Admin](https://cloud.google.com/iam/docs/understanding-roles#serviceusage.serviceUsageAdmin) (`roles/serviceusage.serviceUsageAdmin`) to enable necessary APIs

# @markdown 3. Set up a [Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets).
# @markdown - A new bucket will be set up for you to store artifacts used in this notebook.
# @markdown - [Optional] To use an existing bucket, specify the `gs://` bucket URI. The specified Cloud Storage bucket should be located in the same region as where the notebook was launched. Note that a multi-region bucket (e.g. "us") is not considered a match for a single region covered by the multi-region range (e.g. "us-central1").

BUCKET_URI = ""  # @param {type:"string", placeholder:"[Optional] Cloud Storage bucket URI"}

# The region will be set automatically according to the Colab Enterprise environment.
REGION = ""


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "cxr-foundation")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

## Get online predictions


In [ ]:
# @title #### Import deployed model

# @markdown To get [online predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions), you will need a CXR Foundation [Vertex AI Endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment) that has been deployed from Model Garden. If you have not already done so, go to the [CXR Foundation model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/cxr-foundation) in Model Garden and click "Deploy" to deploy the model.

# @markdown This section gets the Vertex AI Endpoint resource that you deployed from Model Garden to use for online predictions.

# @markdown Fill in the endpoint ID and region below. You can find your deployed endpoint on the [Vertex AI online prediction page](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).

ENDPOINT_ID = ""  # @param {type: "string", placeholder:"e.g. 123456789"}
ENDPOINT_REGION = ""  # @param {type: "string", placeholder:"e.g. us-central1"}

endpoints["endpoint"] = aiplatform.Endpoint(
    endpoint_name=ENDPOINT_ID,
    project=PROJECT_ID,
    location=ENDPOINT_REGION,
)

### Predict

You can send [online prediction requests](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions#predict-request) to the endpoint with chest X-ray images or text to generate embeddings.

The following examples illustrate how to use CXR Foundation to generate:

* Image embeddings from
  * An image or a DICOM file stored in [Cloud Storage](https://cloud.google.com/storage/docs)
  * A DICOM instance stored in [Cloud Healthcare DICOM Store](https://cloud.google.com/healthcare-api/docs/how-tos/dicom)
  * Base64-encoded image bytes
* Text embeddings from a text query


In [ ]:
# @title #### Generate an embedding from an image or DICOM file in Cloud Storage

# @markdown This section shows an example of generating an embedding using an image from the [CXR Foundation Demo Cloud Storage bucket](https://console.cloud.google.com/storage/browser/cxr-foundation-demo), which contains a subset of the NIH Chest X-ray14 dataset [[1]](#scrollTo=k3asO-u59c9Q).

# @markdown The prediction request instance contains the following fields:
# @markdown - `gcs_uri`: `gs://` URI specifying the location of an image or DICOM file stored in Cloud Storage
# @markdown - `bearer_token`: Bearer token used to access data in Cloud Storage (optional for public buckets)

# @markdown You can replace `GCS_URI` below to use your own data.

# @markdown Click "Show Code" to see more details.

GCS_URI = "gs://cxr-foundation-demo/cxr14/inputs/00000001_000.dcm"  # @param {type:"string", placeholder:"Cloud Storage file URI"}

bearer_token = ! gcloud auth print-access-token
bearer_token = bearer_token[0]
creds = credentials.Credentials(token=bearer_token)

_, file_ext = os.path.splitext(GCS_URI)
if file_ext == ".dcm":
    # Displaying a DICOM image is not supported
    pass
else:
    creds = credentials.Credentials(token=bearer_token)
    img = Image.open(io.BytesIO(download_gcs_image_bytes(GCS_URI, creds))).convert(
        "L"
    )  # Convert to grayscale
    display(img)

instances = [
    {
        "gcs_uri": GCS_URI,
        "bearer_token": bearer_token,
    },
]


response = endpoints["endpoint"].predict(instances=instances)
predictions = response.predictions

img_embedding = np.array(predictions[0]["contrastive_img_emb"])

print("Embedding shape: ", img_embedding.shape)

In [ ]:
# @title #### Generate an embedding from a DICOM instance

# @markdown This section shows an example of generating an embedding using a DICOM
# @markdown instance stored in a Cloud Healthcare DICOM Store.

# @markdown The prediction request instance contains the following fields:
# @markdown - `dicomweb_uri`: [DICOMweb](https://cloud.google.com/healthcare-api/docs/how-tos/dicomweb) URI specifying the location of a DICOM instance stored in DICOM store
# @markdown - `bearer_token`: Bearer token used to access data in DICOM store (optional for public data)

# @markdown A sample DICOM instance is not provided. Specify `DICOMWEB_URI` below to use your own data.

# @markdown Click "Show Code" to see more details.

DICOMWEB_URI = "https://healthcare.googleapis.com/v1/projects/PROJECT_ID/locations/LOCATION/datasets/DATASET_ID/dicomStores/DICOM_STORE_ID/dicomWeb/studies/STUDY_INSTANCE_UID/series/SERIES_INSTANCE_UID/instances/INSTANCE_UID"  # @param {type:"string", placeholder:"DICOM instance URI"}

bearer_token = ! gcloud auth print-access-token
bearer_token = bearer_token[0]

instances = [
    {
        "dicomweb_uri": DICOMWEB_URI,
        "bearer_token": bearer_token,
    },
]

response = endpoints["endpoint"].predict(instances=instances)
predictions = response.predictions

img_embedding = np.array(predictions[0]["contrastive_img_emb"])

print("Embedding shape: ", img_embedding.shape)

In [ ]:
# @title #### Generate an embedding from image bytes

# @markdown This section shows an example of generating an embedding using a chest X-ray image from Wikimedia Commons [[2]](#scrollTo=k3asO-u59c9Q) that is downloaded as image bytes.

# @markdown The prediction request instance contains the following field:
# @markdown - `input_bytes`: Base-64 encoded image bytes

# @markdown **Note:** Prediction requests to public endpoints (e.g. endpoints deployed from Model Garden) [must be 1.5 MB or smaller](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions#send-request), so this example uses a lower resolution image.

# @markdown Click "Show Code" to see more details.

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Chest_Xray_PA_3-8-2010.png/1024px-Chest_Xray_PA_3-8-2010.png"
! wget -nc -q {image_url}
img = Image.open("1024px-Chest_Xray_PA_3-8-2010.png").convert(
    "L"
)  # Convert to grayscale
display(img)
instances = [{"input_bytes": common_util.image_to_base64(img, "PNG")}]

response = endpoints["endpoint"].predict(instances=instances)
predictions = response.predictions

img_embedding = np.array(predictions[0]["contrastive_img_emb"])

print("Embedding shape: ", img_embedding.shape)

In [ ]:
# @title #### Generate an embedding from a text query

# @markdown This section shows an example of generating an embedding from a text query.

# @markdown The prediction request instance contains the following field:
# @markdown - `prompt_query`: Text query string

# @markdown Click "Show Code" to see more details.

PROMPT_QUERY = "Airspace opacity"  # @param {type:"string", placeholder:"Text query string"}

instances = [{"prompt_query": PROMPT_QUERY}]

response = endpoints["endpoint"].predict(instances=instances)
predictions = response.predictions

txt_embedding = np.array(predictions[0]["contrastive_txt_emb"])

print("Embedding shape: ", txt_embedding.shape)

## Get batch predictions

In [ ]:
# @title Upload model to Vertex AI Model Registry

# @markdown To get [batch predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions), you must first upload the prebuilt CXR Foundation model to [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction). Batch predictions are made directly on a model in Model Registry without deploying to an endpoint.

MODEL_ID = "cxr-foundation"
MODEL_ARTIFACT_URI = "gs://vertex-model-garden-restricted-us/cxr-foundation"

# The pre-built serving docker image.
SERVE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/health-ai-cxr-foundation.cpu.1-0.ubuntu2004.py312.tf218:20241118-1537-rc0"


def upload_model(model_name: str, artifact_uri: str) -> aiplatform.Model:
    model = aiplatform.Model.upload(
        display_name=model_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
    )
    return model


models["model"] = upload_model(
    model_name=common_util.get_job_name_with_datetime(prefix=MODEL_ID),
    artifact_uri=MODEL_ARTIFACT_URI,
)

### Predict

You can send [batch prediction requests](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#request_a_batch_prediction) to the model using a [JSON Lines](https://jsonlines.org/) file to specify a list of input instances with chest X-ray images or text to generate embeddings.

The following examples illustrate how to use CXR Foundation to generate:

* Image embeddings in batch from
  * An image or a DICOM file stored in [Cloud Storage](https://cloud.google.com/storage/docs)
  * A DICOM instance stored in [Cloud Healthcare DICOM Store](https://cloud.google.com/healthcare-api/docs/how-tos/dicom)
  * Base64-encoded image bytes
* Text embeddings in batch from a text query

In [ ]:
# @title #### Generate embeddings in batch from image or DICOM files in Cloud Storage

# @markdown This section shows an example of generating embeddings in batch using images from the [CXR Foundation Demo Cloud Storage bucket](https://console.cloud.google.com/storage/browser/cxr-foundation-demo), which contains a subset of the NIH Chest X-ray14 dataset [[1]](#scrollTo=k3asO-u59c9Q).

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains the following field:
# @markdown - `gcs_uri`: `gs://` URI specifying the location of an image file stored in Cloud Storage

# @markdown You can replace `GCS_URIS` below to use your own data.

# @markdown **Note:** The custom service account used to launch the batch prediction job must have permission to read the data from Cloud Storage.

# @markdown Click "Show Code" to see more details.

# Comma-separated list of Cloud Storage URIs
GCS_URIS = "gs://cxr-foundation-demo/cxr14/inputs/00000001_000.dcm,gs://cxr-foundation-demo/cxr14/inputs/00000001_001.dcm"  # @param {type:"string", placeholder:"Comma-separated list of Cloud Storage file URIs"}

gcs_uris_list = GCS_URIS.split(",")
batch_predict_instances = [{"gcs_uri": uri} for uri in gcs_uris_list]

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "gcs_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {STAGING_BUCKET}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

gcs_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        STAGING_BUCKET, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(STAGING_BUCKET, batch_predict_prefix, "output"),
    machine_type="n1-highmem-8",
    service_account=SERVICE_ACCOUNT,
)

gcs_batch_predict_job.wait()

print(gcs_batch_predict_job.display_name)
print(gcs_batch_predict_job.resource_name)
print(gcs_batch_predict_job.state)

In [ ]:
# @title #### Generate embeddings in batch from DICOM instances

# @markdown This section shows an example of generating embeddings in batch using DICOM instances stored in a Cloud Healthcare DICOM Store.

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains the following field:
# @markdown - `dicomweb_uri`: [DICOMweb](https://cloud.google.com/healthcare-api/docs/how-tos/dicomweb) URI specifying the location of a DICOM instance stored in DICOM store

# @markdown Sample DICOM instances are not provided. Specify `DICOMWEB_URIS` below to use your own data.

# @markdown **Note:** The custom service account used to launch the batch prediction job must have permission to read the data from DICOM store.

# @markdown Click "Show Code" to see more details.

# Comma-separated list of DICOMweb URIs
DICOMWEB_URIS = "https://healthcare.googleapis.com/v1/projects/PROJECT_ID/locations/LOCATION/datasets/DATASET_ID/dicomStores/DICOM_STORE_ID/dicomWeb/studies/STUDY_INSTANCE_UID/series/SERIES_INSTANCE_UID/instances/INSTANCE_UID"  # @param {type:"string", placeholder:"Comma-separated list of DICOM instance URIs"}

dicomweb_uris_list = DICOMWEB_URIS.split(",")
batch_predict_instances = [{"dicomweb_uri": uri} for uri in dicomweb_uris_list]

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "dicom_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {STAGING_BUCKET}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

dicom_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        STAGING_BUCKET, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(STAGING_BUCKET, batch_predict_prefix, "output"),
    machine_type="n1-highmem-8",
    service_account=SERVICE_ACCOUNT,
)

dicom_batch_predict_job.wait()

print(dicom_batch_predict_job.display_name)
print(dicom_batch_predict_job.resource_name)
print(dicom_batch_predict_job.state)

In [ ]:
# @title #### Generate embeddings in batch from image bytes

# @markdown This section shows an example of generating embeddings in batch using a chest X-ray image from Wikimedia Commons [[2]](#scrollTo=k3asO-u59c9Q) that is downloaded as image bytes.

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains the following field:
# @markdown - `input_bytes`: Base-64 encoded image bytes

# @markdown Click "Show Code" to see more details.

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Chest_Xray_PA_3-8-2010.png/1024px-Chest_Xray_PA_3-8-2010.png"
]
batch_predict_instances = []
for image_url in image_urls:
    ! wget -nc -q {image_url}
    img = Image.open("1024px-Chest_Xray_PA_3-8-2010.png").convert(
        "L"
    )  # Convert to grayscale
    batch_predict_instances.append(
        {"input_bytes": common_util.image_to_base64(img, "PNG")}
    )

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "bytes_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {STAGING_BUCKET}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

bytes_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        STAGING_BUCKET, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(STAGING_BUCKET, batch_predict_prefix, "output"),
    machine_type="n1-highmem-8",
    service_account=SERVICE_ACCOUNT,
)

bytes_batch_predict_job.wait()

print(bytes_batch_predict_job.display_name)
print(bytes_batch_predict_job.resource_name)
print(bytes_batch_predict_job.state)

In [ ]:
# @title #### Generate embeddings in batch from text queries

# @markdown This section shows an example of generating embeddings in batch from text queries.

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains the following field:
# @markdown - `prompt_query`: Text query string

# @markdown Click "Show Code" to see more details.

PROMPT_QUERIES = "Airspace opacity"  # @param {type:"string", placeholder:"Comma-separated list of text query strings"}

prompt_queries_list = PROMPT_QUERIES.split(",")
batch_predict_instances = [{"prompt_query": query} for query in prompt_queries_list]

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "text_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {STAGING_BUCKET}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

text_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        STAGING_BUCKET, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(STAGING_BUCKET, batch_predict_prefix, "output"),
    machine_type="n1-highmem-8",
    service_account=SERVICE_ACCOUNT,
)

text_batch_predict_job.wait()

print(text_batch_predict_job.display_name)
print(text_batch_predict_job.resource_name)
print(text_batch_predict_job.state)

In [ ]:
# @title #### Get prediction results

# @markdown Once the batch prediction job succeeds, you can [retrieve the results](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#retrieve_batch_prediction_results)
# @markdown from the JSON Lines file(s) in the output Cloud Storage location.

# @markdown Click "Show Code" to see more details.


def download_gcs_files_as_json(gcs_files_prefix):
    """Download specified files from Cloud Storage and convert content to JSON."""
    lines = []
    client = storage.Client()
    bucket = storage.bucket.Bucket.from_string(BUCKET_URI, client)
    blobs = bucket.list_blobs(prefix=gcs_files_prefix)
    for blob in blobs:
        with blob.open("r") as f:
            for line in f:
                lines.append(json.loads(line))
    return lines


# Get results from the first batch prediction job (with Cloud Storage image inputs)
# You can replace these variable to get results from another batch prediction job
batch_predict_job = gcs_batch_predict_job
output_embedding_key = "contrastive_img_emb"

batch_predict_output_dir = batch_predict_job.output_info.gcs_output_directory
batch_predict_output_files_prefix = os.path.join(
    batch_predict_output_dir.replace(f"{BUCKET_URI}/", ""), "prediction.results"
)
batch_predict_results = download_gcs_files_as_json(
    gcs_files_prefix=batch_predict_output_files_prefix
)

# Display first two batch prediction results
for i, line in enumerate(batch_predict_results[:2]):
    embedding = np.array(line["prediction"][output_embedding_key])

    print(f"Embedding shape {i}: ", embedding.shape)

## Image attributions

[1] The NIH Chest X-ray14 dataset consists of over 100,000 de-identified images of chest x-rays, with fourteen common disease labels, text-mined from the text radiological reports via NLP techniques.The dataset is available on the [NIH download site](https://nihcc.app.box.com/v/ChestXray-NIHCC) and on [Google Cloud](https://cloud.google.com/healthcare-api/docs/resources/public-datasets/nih-chest).


[2] Stillwaterising, CC0, via Wikimedia Commons

## Next steps

Explore the other [notebooks](https://github.com/google-health/cxr-foundation/blob/master/notebooks) to learn what else you can do with the model.

## Clean up resources

In [ ]:
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME